Contrastive Model
Using KC's pretrained weights or make my own

Look at old contrastive_10_04.py

In [1]:
!pip install --upgrade keras-cv tensorflow
!pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [2]:
import tensorflow as tf
from tensorflow import keras
import keras_cv
from keras_cv import layers as cv_layers
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import glob
import cv2
import re
import os

import PIL
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
tf.keras.backend.clear_session()
tf.config.list_physical_devices('GPU')
devices = tf.config.experimental.list_physical_devices('GPU')
print(devices)
for gpu in devices:
  tf.config.experimental.set_memory_growth(gpu, True)
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])


[]


In [4]:
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 16 # Change this to the maximum which fits
EPOCHS = 50 # 50 might work here too
CROPS_TO = 224
SEED = 26
DIM = (224, 224)
LR = 0.0001
WEIGHT_DECAY = 0.0005

**TODO**




1.   Get current project weights from KC
2.   Run on my current preprocessing data
3.   Quesiton: do I need to geometric augmentations as well
4.   Get encoder look at KCs contrastive learning
5.   Train model and results






In [5]:
# Connect to Drive folder with Data
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/Sarah_Master_Project/Data/Colab_data'

Mounted at /content/drive


In [6]:
os.listdir(folder_path)

['clustered_geopoints.txt',
 'Kenya2019_array_augmented.npy',
 'Kenya_Features.xlsx']

In [7]:
# Load features
feature_file = os.path.join(folder_path, 'Kenya_Features.xlsx')
features = pd.read_excel(feature_file, sheet_name='Kenya_Sarah')



In [8]:
print(features['filename'][0])


features['image_file'] = features['filename'].str.extract(r'(\d+)-')
features.head(5)

225724-IMAGE_RGB_083_224x224_620dd0_330dd0_34dd166580999999994_0dd1249759499999996_133dd0_224x224.png


,filename,country,year,lat,lon,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511,image_file
0,225724-IMAGE_RGB_083_224x224_620dd0_330dd0_34d...,Kenya,1995,0.08759,34.13672,0.000188,0.005483,0.007677,0.015006,0.000314,...,0.000693,0.012207,0.009334,0.004409,0.000395,0.000810,0.002814,0.000109,0.000602,225724
1,225724-IMAGE_RGB_083_224x224_620dd0_330dd0_34d...,Kenya,1995,0.09460,34.13540,0.000188,0.005483,0.007677,0.015006,0.000314,...,0.000693,0.012207,0.009334,0.004409,0.000395,0.000810,0.002814,0.000109,0.000602,225724
2,226291-IMAGE_RGB_083_224x224_621dd0_330dd0_34d...,Kenya,1995,0.08920,34.27900,0.000184,0.005259,0.008056,0.014964,0.000323,...,0.000762,0.012057,0.009620,0.004366,0.000516,0.000725,0.002409,0.000117,0.000704,226291
3,226292-IMAGE_RGB_083_224x224_621dd0_331dd0_34d...,Kenya,1995,0.06570,34.27190,0.000155,0.005272,0.007802,0.014368,0.000305,...,0.000846,0.011664,0.009548,0.004053,0.000475,0.000682,0.002362,0.000120,0.000692,226292
4,226857-IMAGE_RGB_083_224x224_622dd0_329dd0_34d...,Kenya,1995,0.19900,34.36700,0.000135,0.005461,0.007657,0.015523,0.000114,...,0.000597,0.011323,0.009497,0.004475,0.000355,0.000459,0.002560,0.000050,0.000701,226857


In [9]:
image_file = os.path.join(folder_path, 'Kenya2019_array_augmented.npy')
images = np.load(image_file, allow_pickle=True)
images = images.item()


In [10]:
df = pd.read_csv(os.path.join(folder_path, 'clustered_geopoints.txt'), sep = '\t')
#labels = df['avg_wealthscore'].values
df['avg_roofcat'] = df['avg_roofcat'].replace(0.5, 0)
labels = df['avg_roofcat'].values
image_codes = df['image_code'].astype(str).values

#Remove nan
non_nan = ~np.isnan(labels)
labels = labels[non_nan]
image_codes = image_codes[non_nan]

In [11]:
df['avg_roofcat'].value_counts()

,count
avg_roofcat,
1.0,2190
0.0,436


In [12]:
# merge df with features on long and lat

import geopandas as gpd
from shapely.geometry import Point


In [13]:
feature_geopoints = gpd.GeoDataFrame(features, geometry=gpd.points_from_xy(features.lon, features.lat))
df_geopoints = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
df_geopoints = df_geopoints.drop(columns=['index_right'], errors='ignore')

df_merge = gpd.sjoin_nearest(
 df_geopoints, feature_geopoints, how='left', distance_col='distance' )

df_merge = df_merge.loc[~df_merge.index.duplicated(keep='first')]


In [14]:
df_merge.head()

,latitude,longitude,household_count,avg_roofcat,avg_wealthscore,geometry_survey,image_code,lon_left,lat_left,distance,...,feature_503,feature_504,feature_505,feature_506,feature_507,feature_508,feature_509,feature_510,feature_511,image_file
0,-0.433510,37.790243,16,1.0,NaN,POINT (37.790243 -0.43351),1532730,37.770746,-0.437524,0.046780,...,0.000770,0.012423,0.009788,0.003820,0.000642,0.000691,0.002509,0.000089,0.000859,247735
1,-3.879261,39.514019,23,1.0,NaN,POINT (39.514019 -3.879261),1672295,39.520746,-3.895856,0.025154,...,0.000950,0.017742,0.013166,0.006066,0.000515,0.001812,0.003682,0.000111,0.001659,256454
2,-0.048096,35.962199,23,1.0,NaN,POINT (35.962199 -0.048096),1517558,35.979080,-0.062524,0.030592,...,0.000676,0.012992,0.008577,0.003378,0.000676,0.001160,0.002267,0.000186,0.001061,237760
3,0.281347,34.573323,22,1.0,NaN,POINT (34.573323 0.281347),1504076,34.562414,0.270809,0.053517,...,0.000688,0.011422,0.009388,0.004325,0.000355,0.000497,0.002669,0.000077,0.000537,227989
4,-1.987995,37.362238,20,1.0,NaN,POINT (37.362238 -1.987995),1594917,37.354080,-1.979190,0.010137,...,0.000585,0.013972,0.009880,0.003502,0.000877,0.001075,0.002502,0.000173,0.001206,245559


In [15]:
len(df_merge)

2626

In [16]:

df_merge['avg_roofcat'] = df_merge['avg_roofcat'].replace(0.5, 0)
labels = df_merge['avg_roofcat'].values
image_codes = df_merge['image_code'].astype(str).values

#Remove nan
non_nan = ~np.isnan(labels)
labels = labels[non_nan]
image_codes = image_codes[non_nan]

In [17]:
print(len(image_codes))
print(len(labels))

2626
2626


In [18]:
X_images = np.array([images[code] for code in image_codes if code in images])
#X_features = df_merge.loc[:, df_merge.columns.str.startswith('feature_')].values
X_features = df_merge.loc[:, df_merge.columns.str.startswith('feature_')].values.astype('float32')

# Labels should be in the 'labels' variable
# labels = labels


X_images = X_images.astype('float32') / 255.0  # Normalize images
labels = labels.astype('int32')

In [19]:
print(len(labels))
print(len(X_images))
print(len(X_features))

2626
2626
2626


In [20]:
print(X_images.shape)
print(X_features.shape)
print(labels.shape)

(2626, 224, 224, 3)
(2626, 512)
(2626,)


In [32]:
unique, frequency = np.unique(labels,
                              return_counts = True)
# print unique values array
print("Unique Values:",
      unique)

# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0 1]
Frequency Values: [ 436 2190]


In [21]:
X_train_images, X_test_images, X_train_features, X_test_features, y_train, y_test = train_test_split(
    X_images, X_features, labels, test_size=0.2, random_state=42)


print(X_train_images.shape)
print(X_test_images.shape)

print(X_train_features.shape)
print(X_test_features.shape)

print(y_train.shape)
print(y_test.shape)

print(len(labels))

(2100, 224, 224, 3)
(526, 224, 224, 3)
(2100, 512)
(526, 512)
(2100,)
(526,)
2626


In [22]:
NUM_CLASSES = 2
BATCH_SIZE = 16
INPUT_DIM_FEATURES = X_features.shape[1]
print(INPUT_DIM_FEATURES)

512


In [42]:
# Preprocess function
def preprocess_data(images, features, labels):

    labels = tf.one_hot(labels, NUM_CLASSES)
    return {"image_input_unique": images, "feature_input_unique": features}, labels


def preprocess_data(images, features, labels):
    # For binary classification, labels should just be 0 or 1, no need for one-hot encoding
    labels = tf.cast(labels, tf.float32)  # Cast to float32 for compatibility
    print(images.shape)
    print(features.shape)
    print(labels.shape)
    return {"image_input_unique": images, "feature_input_unique": features}, labels


def preprocess_data(images, features, labels):
    labels = tf.cast(labels, tf.float32)
    return (images, features), labels  # Return inputs as a tuple


In [43]:
# Create TensorFlow datasets from both images and features
train_dataset = tf.data.Dataset.from_tensor_slices(((X_train_images, X_train_features), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices(((X_test_images, X_test_features), y_test))



train_dataset = train_dataset.batch(BATCH_SIZE).map(
    lambda x_img_feat, y: preprocess_data(x_img_feat[0], x_img_feat[1], y),
    num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

test_dataset = test_dataset.batch(BATCH_SIZE).map(
    lambda x_img_feat, y: preprocess_data(x_img_feat[0], x_img_feat[1], y),
    num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)


In [44]:
print(train_dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 512), dtype=tf.float32, name=None)), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>


In [45]:
for batch in train_dataset.take(1):
    image_input = batch[0]['image_input_unique']  # Extract the image input
    feature_input = batch[0]['feature_input_unique']  # Extract the feature input
    label_input = batch[1]  # Extract the labels

    print("Image input shape:", image_input.shape)  # Should be (batch_size, 224, 224, 3)
    print("Feature input shape:", feature_input.shape)  # Should be (batch_size, 512)
    print("Label shape:", label_input.shape)  # Should be (batch_size,)

TypeError: tuple indices must be integers or slices, not str

In [46]:
from tensorflow import keras

# Constants
NUM_CLASSES = 2  # For binary classification

# Define image input (shape: 224x224x3)
image_input = keras.Input(shape=(224, 224, 3), name="image_input_unique")

# Process images using a CNN
x_image = keras.layers.Conv2D(32, (3, 3), activation='relu')(image_input)
x_image = keras.layers.MaxPooling2D((2, 2))(x_image)
x_image = keras.layers.Conv2D(64, (3, 3), activation='relu')(x_image)
x_image = keras.layers.MaxPooling2D((2, 2))(x_image)
x_image = keras.layers.Flatten()(x_image)
x_image = keras.layers.Dense(128, activation='relu')(x_image)

# Define feature input (512 weightings)
feature_input = keras.Input(shape=(512,), name="feature_input_unique")

# Combine the processed image features with the additional weightings (features)
combined = keras.layers.concatenate([x_image, feature_input])

# Add dense layers after combining both inputs
combined = keras.layers.Dense(64, activation='relu')(combined)
combined = keras.layers.Dense(32, activation='relu')(combined)

# Output layer for binary classification
output = keras.layers.Dense(1, activation='sigmoid')(combined)

# Define the model
model = keras.Model(inputs=[image_input, feature_input], outputs=output)




In [47]:
# Compile the model
model.compile(
    loss='binary_crossentropy',  # Use binary crossentropy for binary classification
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    metrics=['accuracy']
)

# Display model summary
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_input_unique        │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 222, 222, 32)   │            896 │ image_input_unique[0]… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_4           │ (None, 111, 111, 32)   │              0 │ conv2d_4[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 109, 109, 64)   │         18,496 │ max_pooling2d_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 54, 54, 64)     │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 186624)         │              0 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 128)            │     23,888,000 │ flatten_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ feature_input_unique      │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 640)            │              0 │ dense_8[0][0],         │
│ (Concatenate)             │                        │                │ feature_input_unique[… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 64)             │         41,024 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 32)             │          2,080 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 1)              │             33 │ dense_10[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 23,950,529 (91.36 MB)

 Trainable params: 23,950,529 (91.36 MB)

 Non-trainable params: 0 (0.00 B)

In [48]:
from tensorflow.keras.callbacks import Callback


class PrintBatchShapeCallback(Callback):
    def on_train_batch_begin(self, batch, logs=None):
        # Get a batch of data
        input_data = self.model.inputs
        # Print shapes of image and feature inputs
        print("Image input shape:", input_data[0].shape)
        print("Feature input shape:", input_data[1].shape)

print_shape_callback = PrintBatchShapeCallback()


In [49]:
model.fit(
    train_dataset,                # Training data
    validation_data=test_dataset,  # Validation data
    epochs=16,                    # Number of epochs
    verbose=1,                    # Show progress
    callbacks=[print_shape_callback]  # Pass the custom callback
)

Epoch 1/16
Image input shape: (None, 224, 224, 3)
Feature input shape: (None, 512)


/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['image_input_unique', 'feature_input_unique']. Received: the structure of inputs=('*', '*')
  warnings.warn(


Image input shape: (None, 224, 224, 3)
Feature input shape: (None, 512)
  1/132 ━━━━━━━━━━━━━━━━━━━━ 12:32 6s/step - accuracy: 0.8750 - loss: 0.6698Image input shape: (None, 224, 224, 3)
Feature input shape: (None, 512)
  2/132 ━━━━━━━━━━━━━━━━━━━━ 3:21 2s/step - accuracy: 0.8594 - loss: 0.6574 Image input shape: (None, 224, 224, 3)
Feature input shape: (None, 512)
  3/132 ━━━━━━━━━━━━━━━━━━━━ 3:44 2s/step - accuracy: 0.8438 - loss: 0.6497Image input shape: (None, 224, 224, 3)
Feature input shape: (None, 512)
  4/132 ━━━━━━━━━━━━━━━━━━━━ 4:18 2s/step - accuracy: 0.8281 - loss: 0.6457Image input shape: (None, 224, 224, 3)
Feature input shape: (None, 512)
  5/132 ━━━━━━━━━━━━━━━━━━━━ 4:29 2s/step - accuracy: 0.8225 - loss: 0.6377Image input shape: (None, 224, 224, 3)
Feature input shape: (None, 512)
  6/132 ━━━━━━━━━━━━━━━━━━━━ 4:13 2s/step - accuracy: 0.8226 - loss: 0.6265Image input shape: (None, 224, 224, 3)
Feature input shape: (None, 512)
  7/132 ━━━━━━━━━━━━━━━━━━━━ 4:02 2s/step - 

In [50]:
model.evaluate(test_dataset)

33/33 ━━━━━━━━━━━━━━━━━━━━ 14s 438ms/step - accuracy: 0.8539 - loss: 0.3773


[0.3707779347896576, 0.8631178736686707]

In [52]:
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(test_dataset)
y_pred = np.argmax(y_pred, axis=1)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

33/33 ━━━━━━━━━━━━━━━━━━━━ 14s 430ms/step
Mean Absolute Error: 0.8460076045627376


In [53]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred)
print(f'R-squared: {r2}')


R-squared: -5.493827160493828


In [54]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.15      1.00      0.27        81
           1       0.00      0.00      0.00       445

    accuracy                           0.15       526
   macro avg       0.08      0.50      0.13       526
weighted avg       0.02      0.15      0.04       526



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [57]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,